In [32]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0-preview is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [14]:
t = tf.constant([[1, 2, 3], [5,6,7.]])
t.shape

TensorShape([2, 3])

In [17]:
t[:, 1:]

<tf.Tensor: id=23, shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [6., 7.]], dtype=float32)>

In [20]:
tf.constant([[1],[2]]) @ tf.constant([[1, 2]])

<tf.Tensor: id=31, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [2, 4]])>

In [26]:
np_arr1 = np.array([1,2,3.])
t_arr1 = tf.constant(np_arr1, dtype=tf.float32)
t_arr1

<tf.Tensor: id=34, shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [30]:
tf.constant(5, dtype=tf.float32) + tf.constant(5.)

<tf.Tensor: id=39, shape=(), dtype=float32, numpy=10.0>

In [67]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [76]:
housing = fetch_california_housing()
X_train_valid, X_test, y_train_valid, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)


In [77]:
y_train

array([0.628, 1.969, 0.799, ..., 2.201, 1.053, 3.221])

In [78]:
X_train.shape[1:]

(8,)

In [81]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss=huber_fn, optimizer="nadam", metrics=[keras.metrics.MeanAbsoluteError()])

model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 2s 183us/sample - loss: 0.5491 - mean_absolute_error: 0.9043 - val_loss: 0.2284 - val_mean_absolute_error: 0.5222
Epoch 2/10
11610/11610 [==============================] - 1s 99us/sample - loss: 0.2126 - mean_absolute_error: 0.5069 - val_loss: 0.2026 - val_mean_absolute_error: 0.4909
Epoch 3/10
11610/11610 [==============================] - 1s 105us/sample - loss: 0.2030 - mean_absolute_error: 0.4940 - val_loss: 0.1984 - val_mean_absolute_error: 0.4843
Epoch 4/10
11610/11610 [==============================] - 1s 101us/sample - loss: 0.1975 - mean_absolute_error: 0.4862 - val_loss: 0.1942 - val_mean_absolute_error: 0.4798
Epoch 5/10
11610/11610 [==============================] - 1s 102us/sample - loss: 0.1933 - mean_absolute_error: 0.4801 - val_loss: 0.1977 - val_mean_absolute_error: 0.4820
Epoch 6/10
11610/11610 [==============================] - 1s 103us/sample - loss: 0.1905 - m

In [48]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [46]:
housing.target.reshape(-1, 1)

array([[4.526],
       [3.585],
       [3.521],
       ...,
       [0.923],
       [0.847],
       [0.894]])

In [94]:
def get_obj():
    return {"prop3":3, "prop4":4}

obj1 = {"prop1":1, "prop2":2}
obj1["prop2"]

get_obj()["prop3"]

3

In [100]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold, "param1":123}

In [101]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [105]:
model.loss.threshold

2.0

In [107]:
class Cl1:
    def __init__(self):
        self.param = 123
        
obj1 = Cl1()
obj1.param

123

In [115]:
precision = keras.metrics.Precision()
precision([0,1,1], [1,1,1])

<tf.Tensor: id=205298, shape=(), dtype=float32, numpy=0.6666667>

In [116]:
precision([0,0,1], [1,1,1])

<tf.Tensor: id=205345, shape=(), dtype=float32, numpy=0.5>

In [119]:
recall = keras.metrics.Recall()
recall([1,1,1], [0,1,1])

<tf.Tensor: id=205522, shape=(), dtype=float32, numpy=0.6666667>

In [120]:
recall([1,1,1], [0,0,1])

<tf.Tensor: id=205569, shape=(), dtype=float32, numpy=0.5>

In [125]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([3.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([3.], dtype=float32)>]

In [128]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [140]:
h = HuberMetric(2.)
h(tf.constant([[10.]]), tf.constant([[2.]]))
# h(tf.constant([[2.]]), tf.constant([[10.]]))
h(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))

h.result()

<tf.Tensor: id=206008, shape=(), dtype=float32, numpy=7.0>

In [141]:
h.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=21.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>]

In [168]:
@tf.function
def cube(x):
    print("print: ", x)
    return x**3

cube(tf.constant([3.,4]))

print:  Tensor("x:0", shape=(2,), dtype=float32)


<tf.Tensor: id=206165, shape=(2,), dtype=float32, numpy=array([27., 64.], dtype=float32)>

In [161]:
tf_cube = tf.function(cube)
concrete_func = tf_cube.get_concrete_function(tf.constant([2.,1.,0.]))
concrete_func.graph

In [162]:
concrete_func.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'PartitionedCall' type=PartitionedCall>,
 <tf.Operation 'Identity' type=Identity>]

In [163]:
concrete_func.function_def.signature

name: "__inference_cube_206124"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

In [186]:
result = cube(tf.constant([2,1]))

print:  Tensor("x:0", shape=(2,), dtype=int32)


In [190]:
tf.random.uniform(shape=[3,2])

<tf.Tensor: id=206244, shape=(3, 2), dtype=float32, numpy=
array([[0.05073011, 0.96812725],
       [0.8259343 , 0.15556669],
       [0.12372446, 0.0853281 ]], dtype=float32)>

In [191]:
@tf.function(input_signature=[tf.TensorSpec([None, 28,28], tf.float32)])
def shrink(images):
    print("Tracing", images)
    return images[:, ::2, ::2]

In [194]:
img_batch_1 = tf.random.uniform(shape=[100,28,28])
preprocessing_images=shrink(img_batch_1)

In [209]:
model_builtin = keras.models.Sequential([
    keras.layers.Dense(30, input_shape=X_train_scaled.shape[1:]),
#     keras.layers.LayerNormalization(),
    keras.layers.Dense(1)
])

model_builtin.compile(loss="mse", optimizer="nadam", metrics=["mse"])
history=model_builtin.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid))
model_builtin.evaluate(X_test_scaled, y_test)

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 1s 80us/sample - loss: 2.0627 - mse: 2.0627 - val_loss: 1.1070 - val_mse: 1.1070
Epoch 2/10
11610/11610 [==============================] - 0s 43us/sample - loss: 0.6252 - mse: 0.6252 - val_loss: 0.6266 - val_mse: 0.6266
Epoch 3/10
11610/11610 [==============================] - 1s 43us/sample - loss: 0.5401 - mse: 0.5401 - val_loss: 0.5110 - val_mse: 0.5110
Epoch 4/10
11610/11610 [==============================] - 1s 43us/sample - loss: 0.5231 - mse: 0.5231 - val_loss: 0.5208 - val_mse: 0.5208
Epoch 5/10
11610/11610 [==============================] - 1s 44us/sample - loss: 0.5210 - mse: 0.5210 - val_loss: 0.5042 - val_mse: 0.5042
Epoch 6/10
11610/11610 [==============================] - 1s 45us/sample - loss: 0.5257 - mse: 0.5257 - val_loss: 0.5146 - val_mse: 0.5146
Epoch 7/10
11610/11610 [==============================] - 1s 45us/sample - loss: 0.5215 - mse: 0.5215 - val_loss: 0.6

[0.6406431704528572, 0.6406432]

In [409]:
class MyNorm(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
#         print("input_shape", batch_input_shape)
        self.a = self.add_weight(name="a", shape=[batch_input_shape[-1]], initializer="ones")
        self.b = self.add_weight(name="b", shape=[batch_input_shape[-1]], initializer="zeros")
    def call(self, X):
        means, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        deviation = tf.sqrt(variance)
        temp = tf.divide((X - means) , (deviation + 0.001))
        return self.a * temp + self.b
        
        
model_custom = keras.models.Sequential([
    keras.layers.Dense(30, input_shape=X_train_scaled.shape[1:]),
    MyNorm(),
    keras.layers.Dense(1)
])

model_custom.compile(loss="mse", optimizer="nadam")
history=model_custom.fit(X_train_scaled, y_train, epochs=10)        
        

Train on 11610 samples
Epoch 1/10
11610/11610 [==============================] - 2s 215us/sample - loss: 1.3428
Epoch 2/10
11610/11610 [==============================] - 0s 39us/sample - loss: 0.5812
Epoch 3/10
11610/11610 [==============================] - 0s 38us/sample - loss: 0.5136
Epoch 4/10
11610/11610 [==============================] - 0s 41us/sample - loss: 0.4828
Epoch 5/10
11610/11610 [==============================] - 0s 37us/sample - loss: 0.4619
Epoch 6/10
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4433
Epoch 7/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.4274
Epoch 8/10
11610/11610 [==============================] - 0s 35us/sample - loss: 0.4145
Epoch 9/10
11610/11610 [==============================] - 0s 39us/sample - loss: 0.4043
Epoch 10/10
11610/11610 [==============================] - 0s 37us/sample - loss: 0.3966


In [264]:
t = tf.random.uniform(shape=([200, 2,3, 5, 10]))

In [391]:
tf.nn.moments(tf.random.uniform(shape=([3000])), axes=-1, keepdims=True)


(<tf.Tensor: id=300420, shape=(1,), dtype=float32, numpy=array([0.5143874], dtype=float32)>,
 <tf.Tensor: id=300424, shape=(1,), dtype=float32, numpy=array([0.08210342], dtype=float32)>)

In [386]:
tf.random.uniform(shape=([30]))

<tf.Tensor: id=300261, shape=(30,), dtype=float32, numpy=
array([0.27846277, 0.03039241, 0.79354954, 0.76845515, 0.12414634,
       0.7888726 , 0.6834048 , 0.6948552 , 0.9479108 , 0.08152926,
       0.2686304 , 0.38805842, 0.5830244 , 0.80564296, 0.188182  ,
       0.18048859, 0.7982403 , 0.3327682 , 0.63477623, 0.24269676,
       0.53012836, 0.5760546 , 0.7798474 , 0.48397195, 0.8033732 ,
       0.7519386 , 0.97443366, 0.48904908, 0.10616124, 0.6213938 ],
      dtype=float32)>

In [405]:
def ttt(X):
    a = tf.random.uniform(shape=([30]))
    print("a=", a)
    b = tf.random.uniform(shape=([30]))
    print("b=", b)
    means, variance = tf.nn.moments(tf.random.uniform(shape=([3000])), axes=-1, keepdims=True)
    print("means, variance = ", means, variance)
    deviation = tf.sqrt(variance)
    print("deviation = ", deviation)
    
    temp = tf.divide((X - means) , (deviation + 0.001))
    print("temp = ", temp)
    return a * temp + b

ttt(tf.random.uniform(shape=([30])))

a= tf.Tensor(
[0.7316015  0.3580588  0.5641458  0.17306602 0.52180123 0.71826744
 0.40039003 0.34134495 0.3757857  0.19959927 0.17109692 0.07648611
 0.54324865 0.7930169  0.9711796  0.99170196 0.74457157 0.19307542
 0.20291448 0.10726082 0.9425949  0.8075644  0.6143062  0.48760366
 0.40510833 0.62923884 0.9704093  0.58255124 0.80988216 0.47977126], shape=(30,), dtype=float32)
b= tf.Tensor(
[0.00123072 0.7109895  0.52582383 0.8297168  0.39738047 0.887128
 0.07771122 0.89709985 0.5886053  0.3850665  0.05345154 0.5661093
 0.23802102 0.85678697 0.53850746 0.18685782 0.24370658 0.04736054
 0.04004717 0.6054511  0.48595405 0.41821063 0.61758685 0.11897838
 0.6875551  0.38433957 0.47670245 0.5735116  0.96251774 0.91600215], shape=(30,), dtype=float32)
means, variance =  tf.Tensor([0.49628252], shape=(1,), dtype=float32) tf.Tensor([0.08443209], shape=(1,), dtype=float32)
deviation =  tf.Tensor([0.290572], shape=(1,), dtype=float32)
temp =  tf.Tensor(
[-0.08877428 -1.5533848  -0.33071756 -1.394

<tf.Tensor: id=300838, shape=(30,), dtype=float32, numpy=
array([-6.3716680e-02,  1.5478635e-01,  3.3925092e-01,  5.8832908e-01,
        6.3610077e-04,  3.1442690e-01,  6.4867109e-02,  7.3163015e-01,
        1.9086525e-01,  4.7744194e-01,  2.7397284e-01,  6.9397342e-01,
        5.2309394e-01,  1.0927397e-01, -8.9741433e-01, -2.1212202e-01,
       -6.2726527e-02,  2.5504759e-01, -1.4509158e-01,  6.1614370e-01,
        8.7154871e-01,  3.5312378e-01,  7.8175747e-01,  9.0224040e-01,
        4.9639946e-01,  6.6851795e-01, -7.9691088e-01,  1.1060565e+00,
        7.1279472e-01,  7.6240003e-01], dtype=float32)>